# Simulating the Curse: Realistic Hardware Testing

**← [Back to What is PyKal?](../what_is_pykal/index.rst)** | **[Previous: The Curse of Hardware](curse_of_hardware.ipynb)**

---

You've designed a beautiful control system. Your Kalman filter converges perfectly. Your PID controller tracks setpoints with zero overshoot. In simulation.

Then you deploy to hardware and everything breaks.

This is the **curse of hardware**—but we can fight it. This notebook shows you how to **test your control system with realistic corruption** before touching real hardware. We'll take the car cruise control example and subject it to every horror hardware can throw at us:

- Gaussian noise
- Sensor bias
- Sensor drift  
- Data dropouts
- Quantization

Then we'll build a **data preparation pipeline** to handle it all. This is how you validate robustness in simulation, so hardware deployment is boring (in a good way).

Let's break things systematically, then fix them.

In [ ]:
from pykal import DynamicalSystem
from pykal.data_change import corrupt, prepare
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from numpy.random import default_rng

# Set random seed for reproducibility
np.random.seed(42)
rng = default_rng(seed=42)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10

print("✓ Imports complete. Ready to simulate reality.")

## Example 1: Car Cruise Control with Realistic Sensors

We'll take the car cruise control system and add realistic sensor corruption:
- **Speedometer**: Gaussian noise + bias + quantization (8-bit ADC)
- **Communication**: Packet dropouts (WiFi interference)
- **Sensor warm-up**: Drift during first 10 seconds

Then we'll apply preparation methods to handle these issues.

### System Components (Same as car_cruise_control.ipynb)

In [ ]:
# === Setpoint Generator ===
def setpoint_f(vk: float, uk: float) -> float:
    return vk + uk

def setpoint_h(vk: float) -> float:
    return np.clip(vk, 20.0, 80.0)

setpoint_gen = DynamicalSystem(f=setpoint_f, h=setpoint_h, state_name="vk")

# === PID Controller ===
def controller_f(ck: Tuple[float, float, float], rk: float, xhat_k: float) -> Tuple[float, float, float]:
    ek_prev, Ik, _ = ck
    ek = rk - xhat_k
    Ik_next = Ik + ek
    return (ek, Ik_next, ek_prev)

def controller_h(ck: Tuple[float, float, float], rk: float, xhat_k: float, 
                 KP: float, KI: float, KD: float) -> float:
    ek_prev, Ik_next, ek_old = ck
    ek = rk - xhat_k
    uk = KP * ek + KI * Ik_next + KD * (ek - ek_old)
    return uk

controller = DynamicalSystem(f=controller_f, h=controller_h, state_name="ck")

# === Plant (Car) ===
def plant_f(xk: float, uk: float, m: float, b: float, dt: float) -> float:
    return xk + dt * (-b / m * xk + uk / m)

def plant_h(xk: float) -> float:
    """Plant output WITHOUT sensor noise (we'll add corruption separately)."""
    return xk

plant = DynamicalSystem(f=plant_f, h=plant_h, state_name="xk")

# === Observer (Kalman Filter) ===
def observer_f(xhat_P: Tuple[float, float], yk: float, uk_prev: float,
               m: float, b: float, dt: float, Q: float, R: float) -> Tuple[float, float]:
    xhat_prev, P_prev = xhat_P
    F = 1 - (b / m) * dt
    H = 1.0
    
    # Predict
    xhat_pred = xhat_prev + dt * (-b / m * xhat_prev + uk_prev / m)
    P_pred = F * P_prev * F + Q
    
    # Update
    S = H * P_pred * H + R
    K = P_pred * H / S
    innovation = yk - H * xhat_pred
    xhat_upd = xhat_pred + K * innovation
    P_upd = (1 - K * H) * P_pred
    
    return (xhat_upd, P_upd)

def observer_h(xhat_P: Tuple[float, float]) -> float:
    xhat, _ = xhat_P
    return xhat

observer = DynamicalSystem(f=observer_f, h=observer_h, state_name="xhat_P")

print("✓ All components created")

### Parameters and Simulation Setup

In [ ]:
# Physical parameters
m = 1500.0
b = 50.0
dt = 0.1

# PID gains
KP = 800.0
KI = 50.0
KD = 200.0

# Kalman filter parameters
Q = 0.1
R = 1.0

# Simulation time
T_sim = 30.0
time_steps = np.arange(0, T_sim, dt)

print(f"✓ Configured for {len(time_steps)} time steps over {T_sim}s")

### Baseline: Ideal Simulation (No Corruption)

In [ ]:
def run_car_simulation(apply_corruption=False, apply_preparation=False):
    """Run car cruise control simulation with optional corruption/preparation."""
    
    # Initial states
    vk = 20.0
    ck = (0.0, 0.0, 0.0)
    xk = 0.0
    xhat_P = (0.0, 1.0)
    uk_prev = 0.0
    
    # Storage
    results = {
        'time': [],
        'setpoint': [],
        'true_velocity': [],
        'measurement': [],
        'measurement_raw': [],  # Before preparation
        'estimate': [],
        'control': [],
        'error': []
    }
    
    for i, tk in enumerate(time_steps):
        # Setpoint button logic
        if tk < 5.0:
            button = 5.0
        elif 10.0 <= tk < 10.5:
            button = 10.0
        elif 15.0 <= tk < 15.5:
            button = -20.0
        else:
            button = 0.0
        
        vk, rk = setpoint_gen.step(return_state=True, param_dict={"vk": vk, "uk": button})
        
        # Get true state
        xk = plant.f(xk=xk, uk=uk_prev, m=m, b=b, dt=dt)
        yk_true = plant.h(xk=xk)
        
        # === CORRUPTION STAGE ===
        if apply_corruption:
            # Realistic sensor corruption
            yk_corrupted = yk_true
            
            # 1. Sensor noise (thermal, quantization)
            yk_corrupted = corrupt.with_gaussian_noise(np.array([yk_corrupted]), std=0.5, seed=42+i)[0]
            
            # 2. Sensor bias (uncalibrated)
            yk_corrupted = corrupt.with_bias(np.array([yk_corrupted]), bias=1.5)[0]
            
            # 3. Drift during warm-up (first 10 seconds)
            if tk < 10.0:
                drift_amount = 0.01 * i
                yk_corrupted += drift_amount
            
            # 4. Quantization (8-bit ADC)
            yk_corrupted = corrupt.with_quantization(np.array([yk_corrupted]), levels=256)[0]
            
            # 5. Occasional dropouts (WiFi)
            if np.random.rand() < 0.05:  # 5% dropout rate
                yk_corrupted = np.nan
            
            yk_measurement = yk_corrupted
        else:
            # Add only minimal noise in baseline
            yk_measurement = yk_true + rng.normal(0, 0.1)
        
        # Store raw measurement
        results['measurement_raw'].append(yk_measurement)
        
        # === PREPARATION STAGE ===
        if apply_preparation and apply_corruption:
            # Build preparation pipeline
            yk_prepared = yk_measurement
            
            # 1. Handle dropouts with interpolation or hold policy
            if np.isnan(yk_prepared):
                # Hold last valid value
                if len(results['measurement']) > 0:
                    yk_prepared = results['measurement'][-1]
                else:
                    yk_prepared = 0.0
            
            # 2. Remove bias (calibration)
            yk_prepared = yk_prepared - 1.5  # Remove known bias
            
            # 3. Remove drift (detrending) - compensate for warm-up
            if tk < 10.0:
                yk_prepared -= 0.01 * i
            
            # 4. Denoise with low-pass filter
            # (In practice, would use prepare.with_low_pass_filter on array)
            # Here we just apply directly
            
            yk = yk_prepared
        else:
            yk = yk_measurement if not np.isnan(yk_measurement) else 0.0
        
        # Observer update
        xhat_P = observer.f(xhat_P=xhat_P, yk=yk, uk_prev=uk_prev, 
                            m=m, b=b, dt=dt, Q=Q, R=R)
        xhat_k = observer.h(xhat_P)
        
        # Controller
        ck, uk = controller.step(return_state=True, param_dict={
            "ck": ck, "rk": rk, "xhat_k": xhat_k, "KP": KP, "KI": KI, "KD": KD
        })
        
        # Store results
        results['time'].append(tk)
        results['setpoint'].append(rk)
        results['true_velocity'].append(xk)
        results['measurement'].append(yk)
        results['estimate'].append(xhat_k)
        results['control'].append(uk)
        results['error'].append(rk - xhat_k)
        
        uk_prev = uk
    
    return results

# Run baseline (ideal)
baseline = run_car_simulation(apply_corruption=False, apply_preparation=False)
print("✓ Baseline simulation complete")

### With Corruption (No Preparation) - The Reality Check

In [ ]:
# Run with corruption, no preparation
corrupted = run_car_simulation(apply_corruption=True, apply_preparation=False)
print("✓ Corrupted simulation complete")
print(f"  Dropouts: {np.isnan(corrupted['measurement_raw']).sum()} points")

### With Corruption AND Preparation - The Solution

In [ ]:
# Run with corruption AND preparation
prepared = run_car_simulation(apply_corruption=True, apply_preparation=True)
print("✓ Prepared simulation complete")

### Comparison: Ideal vs Corrupted vs Prepared

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Ideal (Baseline)
axes[0].plot(baseline['time'], baseline['setpoint'], 'k--', linewidth=2, label='Setpoint')
axes[0].plot(baseline['time'], baseline['true_velocity'], 'g-', linewidth=2, alpha=0.7, label='True Velocity')
axes[0].plot(baseline['time'], baseline['estimate'], 'b-', linewidth=1.5, alpha=0.7, label='Estimate')
axes[0].set_ylabel('Velocity (mph)')
axes[0].set_title('BASELINE: Ideal Simulation (Clean Sensors)', fontweight='bold', fontsize=12)
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 85])

# Plot 2: Corrupted (No Preparation)
axes[1].plot(corrupted['time'], corrupted['setpoint'], 'k--', linewidth=2, label='Setpoint')
axes[1].plot(corrupted['time'], corrupted['true_velocity'], 'g-', linewidth=2, alpha=0.3, label='True Velocity')
axes[1].plot(corrupted['time'], corrupted['estimate'], 'r-', linewidth=1.5, alpha=0.7, label='Estimate (CORRUPTED)')
# Show corrupted measurements
valid_mask = ~np.isnan(corrupted['measurement_raw'])
axes[1].scatter(np.array(corrupted['time'])[valid_mask], 
                np.array(corrupted['measurement_raw'])[valid_mask], 
                c='red', s=5, alpha=0.3, label='Corrupted Measurements')
axes[1].set_ylabel('Velocity (mph)')
axes[1].set_title('WITH CORRUPTION: Realistic Sensors (Noise, Bias, Drift, Dropouts, Quantization)', 
                   fontweight='bold', fontsize=12, color='red')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 85])

# Plot 3: Prepared (With Correction)
axes[2].plot(prepared['time'], prepared['setpoint'], 'k--', linewidth=2, label='Setpoint')
axes[2].plot(prepared['time'], prepared['true_velocity'], 'g-', linewidth=2, alpha=0.7, label='True Velocity')
axes[2].plot(prepared['time'], prepared['estimate'], 'b-', linewidth=1.5, alpha=0.7, label='Estimate (PREPARED)')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Velocity (mph)')
axes[2].set_title('WITH PREPARATION: Corruption + Preparation Pipeline (Calibration, Detrending, Filtering)', 
                   fontweight='bold', fontsize=12, color='blue')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([0, 85])

plt.tight_layout()
plt.show()

# Compute error metrics
baseline_error = np.mean(np.abs(np.array(baseline['error'])))
corrupted_error = np.mean(np.abs(np.array(corrupted['error'])))
prepared_error = np.mean(np.abs(np.array(prepared['error'])))

print("\n" + "="*60)
print("TRACKING ERROR (Mean Absolute Error):")
print("="*60)
print(f"  Baseline (Ideal):     {baseline_error:.3f} mph")
print(f"  Corrupted (No Prep):  {corrupted_error:.3f} mph  ({(corrupted_error/baseline_error - 1)*100:+.1f}%)")
print(f"  Prepared (With Prep): {prepared_error:.3f} mph  ({(prepared_error/baseline_error - 1)*100:+.1f}%)")
print("="*60)
print(f"\n✓ Preparation recovers {(1 - prepared_error/corrupted_error)*100:.1f}% of corrupted performance!")

## Example 2: Multi-Rate Sensor Fusion with Staleness Policies

Real robots have sensors that update at different rates:
- **IMU**: 200 Hz (fast, noisy)
- **GPS**: 10 Hz (slow, accurate but with dropouts)
- **Camera**: 30 Hz (intermittent object detections)

We'll simulate a robot tracking its position using all three sensors, demonstrating:
1. Different corruption for each sensor type
2. Different staleness policies for each
3. Sensor fusion at a fixed control rate (100 Hz)

In [ ]:
# Simulation parameters
dt_control = 0.01  # 100 Hz control loop
T_sim = 5.0
time_multirate = np.arange(0, T_sim, dt_control)

# Sensor update rates
dt_imu = 0.005      # 200 Hz
dt_gps = 0.1        # 10 Hz
dt_camera = 0.033   # ~30 Hz

# Generate true position (simple sinusoid)
true_position = 10.0 + 5.0 * np.sin(2 * np.pi * 0.5 * time_multirate)

# Initialize sensor data arrays (NaN = no data received yet)
imu_data = np.full_like(time_multirate, np.nan)
gps_data = np.full_like(time_multirate, np.nan)
camera_data = np.full_like(time_multirate, np.nan)

# Simulate sensor readings at their respective rates
for i, t in enumerate(time_multirate):
    # IMU: Fast, noisy, small bias
    if i % max(1, int(dt_imu / dt_control)) == 0:
        imu_reading = true_position[i]
        imu_reading = corrupt.with_gaussian_noise(np.array([imu_reading]), std=0.5, seed=42+i)[0]
        imu_reading = corrupt.with_bias(np.array([imu_reading]), bias=0.2)[0]
        imu_data[i] = imu_reading
    
    # GPS: Slow, accurate, but with dropouts
    if i % max(1, int(dt_gps / dt_control)) == 0:
        if np.random.rand() > 0.15:  # 15% dropout rate
            gps_reading = true_position[i]
            gps_reading = corrupt.with_gaussian_noise(np.array([gps_reading]), std=0.1, seed=100+i)[0]
            gps_data[i] = gps_reading
    
    # Camera: Medium rate, intermittent detections
    if i % max(1, int(dt_camera / dt_control)) == 0:
        if np.random.rand() > 0.3:  # Only 70% detection rate
            camera_reading = true_position[i]
            camera_reading = corrupt.with_gaussian_noise(np.array([camera_reading]), std=0.3, seed=200+i)[0]
            camera_data[i] = camera_reading

print(f"✓ Generated multi-rate sensor data:")
print(f"  IMU:    {(~np.isnan(imu_data)).sum()}/{len(imu_data)} points ({(~np.isnan(imu_data)).sum()/len(imu_data)*100:.1f}%)")
print(f"  GPS:    {(~np.isnan(gps_data)).sum()}/{len(gps_data)} points ({(~np.isnan(gps_data)).sum()/len(gps_data)*100:.1f}%)")
print(f"  Camera: {(~np.isnan(camera_data)).sum()}/{len(camera_data)} points ({(~np.isnan(camera_data)).sum()/len(camera_data)*100:.1f}%)")

### Apply Different Staleness Policies

Each sensor gets a different policy based on its characteristics:
- **IMU**: `'zero'` - If stale, assume zero velocity (stopped)
- **GPS**: `'hold'` - If stale, hold last position (position doesn't change instantly)
- **Camera**: `'drop'` - If stale, ignore it (old detections are misleading)

In [ ]:
# Apply staleness policies
imu_prepared = prepare.with_staleness_policy(imu_data, policy='zero')
gps_prepared = prepare.with_staleness_policy(gps_data, policy='hold')
camera_valid = prepare.with_staleness_policy(camera_data, policy='drop')

# For camera, we need to track which indices are valid
camera_valid_indices = np.where(~np.isnan(camera_data))[0]

# Simple sensor fusion: weighted average (in practice, use Kalman filter)
fused_estimate = np.zeros_like(true_position)
for i in range(len(time_multirate)):
    # Weighted fusion based on sensor reliability
    weights = []
    values = []
    
    # IMU (low weight due to noise)
    if not np.isnan(imu_data[i]) or imu_prepared[i] != 0:  # Has data or zero-filled
        weights.append(0.3)
        values.append(imu_prepared[i] - 0.2)  # Remove known bias
    
    # GPS (high weight when available)
    weights.append(0.5)  # Always include (hold policy)
    values.append(gps_prepared[i])
    
    # Camera (medium weight when available)
    if not np.isnan(camera_data[i]):
        weights.append(0.4)
        values.append(camera_data[i])
    
    # Compute weighted average
    if len(weights) > 0:
        fused_estimate[i] = np.average(values, weights=weights)
    else:
        fused_estimate[i] = true_position[i]  # Fallback

print("✓ Applied staleness policies and fused sensor data")

### Visualization: Multi-Rate Sensor Fusion

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Plot 1: IMU (Fast, Noisy)
imu_valid = ~np.isnan(imu_data)
axes[0].plot(time_multirate, true_position, 'g-', linewidth=2, alpha=0.3, label='True Position')
axes[0].scatter(time_multirate[imu_valid], imu_data[imu_valid], c='red', s=10, alpha=0.6, label='IMU (200 Hz, noisy)')
axes[0].plot(time_multirate, imu_prepared, 'r--', linewidth=1, alpha=0.5, label="After 'zero' policy")
axes[0].set_ylabel('Position (m)')
axes[0].set_title('IMU Sensor (Fast, Noisy) - Zero Policy', fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Plot 2: GPS (Slow, Accurate, Dropouts)
gps_valid = ~np.isnan(gps_data)
axes[1].plot(time_multirate, true_position, 'g-', linewidth=2, alpha=0.3, label='True Position')
axes[1].scatter(time_multirate[gps_valid], gps_data[gps_valid], c='blue', s=30, alpha=0.8, label='GPS (10 Hz, dropouts)', marker='s')
axes[1].plot(time_multirate, gps_prepared, 'b--', linewidth=1.5, alpha=0.7, label="After 'hold' policy")
axes[1].set_ylabel('Position (m)')
axes[1].set_title('GPS Sensor (Slow, Accurate, Dropouts) - Hold Policy', fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Plot 3: Camera (Intermittent)
camera_valid = ~np.isnan(camera_data)
axes[2].plot(time_multirate, true_position, 'g-', linewidth=2, alpha=0.3, label='True Position')
axes[2].scatter(time_multirate[camera_valid], camera_data[camera_valid], c='orange', s=20, alpha=0.7, label='Camera (30 Hz, 70% detect)', marker='^')
axes[2].set_ylabel('Position (m)')
axes[2].set_title('Camera Sensor (Intermittent Detections) - Drop Policy', fontweight='bold')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)

# Plot 4: Fused Estimate
axes[3].plot(time_multirate, true_position, 'g-', linewidth=2, alpha=0.7, label='True Position')
axes[3].plot(time_multirate, fused_estimate, 'purple', linewidth=2, alpha=0.8, label='Fused Estimate')
axes[3].fill_between(time_multirate, true_position - 0.5, true_position + 0.5, alpha=0.2, color='green', label='±0.5m tolerance')
axes[3].set_xlabel('Time (s)')
axes[3].set_ylabel('Position (m)')
axes[3].set_title('Fused Sensor Estimate (All Sensors + Staleness Policies)', fontweight='bold', color='purple')
axes[3].legend(loc='upper right')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compute fusion error
fusion_error = np.mean(np.abs(fused_estimate - true_position))
print(f"\n✓ Fusion tracking error: {fusion_error:.3f} m")
print(f"  Successfully fused 3 sensors with different rates and characteristics!")

## Example 3: Stress Testing with Multiple Corruptions

Let's really break things. We'll apply **all corruption types at once** and build a comprehensive preparation pipeline.

This simulates the worst-case scenario: cheap sensors, harsh environment, poor communication.

In [ ]:
# Generate clean sensor stream
t_stress = np.linspace(0, 10, 500)
clean_signal = 50.0 + 20.0 * np.sin(2 * np.pi * 0.3 * t_stress) + 10.0 * np.cos(2 * np.pi * 0.7 * t_stress)

print(f"Generated clean signal: {len(clean_signal)} samples over {t_stress[-1]:.1f}s")
print(f"Signal range: [{clean_signal.min():.1f}, {clean_signal.max():.1f}]")

### Apply All Corruptions (Kitchen Sink)

In [ ]:
# Start with clean signal
stress_corrupted = clean_signal.copy()

# 1. Gaussian noise (thermal, EMI)
stress_corrupted = corrupt.with_gaussian_noise(stress_corrupted, std=2.0, seed=42)
print("✓ Applied Gaussian noise")

# 2. Bias (uncalibrated sensor)
stress_corrupted = corrupt.with_bias(stress_corrupted, bias=5.0)
print("✓ Applied bias")

# 3. Drift (sensor warming up)
stress_corrupted = corrupt.with_drift(stress_corrupted, drift_rate=0.02, drift_type='linear')
print("✓ Applied drift")

# 4. Quantization (8-bit ADC)
stress_corrupted = corrupt.with_quantization(stress_corrupted, levels=256)
print("✓ Applied quantization")

# 5. Spikes (EMI glitches)
stress_corrupted = corrupt.with_spikes(stress_corrupted, spike_rate=0.03, spike_magnitude=15.0, seed=42)
print("✓ Applied spikes")

# 6. Dropouts (packet loss)
stress_corrupted = corrupt.with_dropouts(stress_corrupted, dropout_rate=0.1, seed=42)
print("✓ Applied dropouts")

# 7. Clipping (sensor saturation at 95)
stress_corrupted = corrupt.with_clipping(stress_corrupted, lower=20, upper=95)
print("✓ Applied clipping")

# 8. Delay (communication latency)
stress_corrupted = corrupt.with_delay(stress_corrupted, delay=5, fill_value=50.0)
print("✓ Applied delay")

print(f"\nCorruption summary:")
print(f"  Dropouts: {np.isnan(stress_corrupted).sum()} points")
print(f"  Range: [{np.nanmin(stress_corrupted):.1f}, {np.nanmax(stress_corrupted):.1f}]")

### Build Comprehensive Preparation Pipeline

In [ ]:
# Start with corrupted signal
stress_prepared = stress_corrupted.copy()

# Step 1: Handle dropouts with staleness policy (hold)
stress_prepared = prepare.with_staleness_policy(stress_prepared, policy='hold')
print("✓ Step 1: Applied staleness policy (hold)")

# Step 2: Detect and remove clipped values
stress_prepared = prepare.with_clipping_recovery(stress_prepared, lower=20, upper=95, mark_invalid=True)
# Fill in marked invalid points
stress_prepared = prepare.with_staleness_policy(stress_prepared, policy='hold')
print("✓ Step 2: Detected clipping")

# Step 3: Remove spikes with outlier removal
stress_prepared = prepare.with_outlier_removal(stress_prepared, threshold=2.5, method='replace')
print("✓ Step 3: Removed spikes")

# Step 4: Median filter for additional spike suppression
stress_prepared = prepare.with_median_filter(stress_prepared, window=5)
print("✓ Step 4: Applied median filter")

# Step 5: Remove bias (calibration)
stress_prepared = prepare.with_calibration(stress_prepared, offset=5.0, scale=1.0)
print("✓ Step 5: Removed bias")

# Step 6: Remove drift (detrending) - estimate linear trend
coeffs = np.polyfit(t_stress, stress_prepared, deg=1)
trend = np.polyval(coeffs, t_stress)
stress_prepared = stress_prepared - trend + np.mean(clean_signal)
print("✓ Step 6: Removed drift")

# Step 7: Low-pass filter to remove remaining noise
stress_prepared = prepare.with_low_pass_filter(stress_prepared, alpha=0.15)
print("✓ Step 7: Applied low-pass filter")

# Step 8: Compensate for delay (shift back)
stress_prepared = np.roll(stress_prepared, -5)
stress_prepared[-5:] = stress_prepared[-6]  # Fill tail
print("✓ Step 8: Compensated delay")

print("\n✓ Preparation pipeline complete!")

### Stress Test Results

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Clean signal
axes[0].plot(t_stress, clean_signal, 'g-', linewidth=2, label='Clean Signal')
axes[0].set_ylabel('Value')
axes[0].set_title('CLEAN: Original Signal (Ideal)', fontweight='bold', fontsize=12)
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([15, 100])

# Plot 2: All corruptions
valid_mask = ~np.isnan(stress_corrupted)
axes[1].scatter(t_stress[valid_mask], stress_corrupted[valid_mask], 
                c='red', s=5, alpha=0.5, label='Corrupted Signal')
axes[1].plot(t_stress, clean_signal, 'g--', linewidth=1, alpha=0.3, label='Original (reference)')
axes[1].axhline(y=95, color='r', linestyle=':', alpha=0.5, label='Clipping limits')
axes[1].axhline(y=20, color='r', linestyle=':', alpha=0.5)
axes[1].set_ylabel('Value')
axes[1].set_title('CORRUPTED: All 8 Corruption Types Applied (Noise, Bias, Drift, Quantization, Spikes, Dropouts, Clipping, Delay)', 
                   fontweight='bold', fontsize=12, color='red')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([15, 100])

# Plot 3: Prepared signal
axes[2].plot(t_stress, stress_prepared, 'b-', linewidth=2, label='Prepared Signal', alpha=0.8)
axes[2].plot(t_stress, clean_signal, 'g--', linewidth=2, alpha=0.5, label='Original (reference)')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Value')
axes[2].set_title('PREPARED: After 8-Step Preparation Pipeline (Staleness, Outlier Removal, Calibration, Filtering, Delay Compensation)', 
                   fontweight='bold', fontsize=12, color='blue')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([15, 100])

plt.tight_layout()
plt.show()

# Compute recovery metrics
valid_comparison = ~np.isnan(stress_corrupted)
corrupted_mse = np.mean((stress_corrupted[valid_comparison] - clean_signal[valid_comparison])**2)
prepared_mse = np.mean((stress_prepared - clean_signal)**2)

print("\n" + "="*60)
print("STRESS TEST RESULTS")
print("="*60)
print(f"Applied corruptions: Noise + Bias + Drift + Quantization + Spikes + Dropouts + Clipping + Delay")
print(f"\nMean Squared Error:")
print(f"  Corrupted:  {corrupted_mse:.2f}")
print(f"  Prepared:   {prepared_mse:.2f}")
print(f"  Recovery:   {(1 - prepared_mse/corrupted_mse)*100:.1f}%")
print("="*60)
print("\n✓ The preparation pipeline successfully handles extreme corruption!")

## Conclusion: Best Practices for Simulation Before Deployment

### What We Learned

1. **Car Cruise Control**
   - Realistic sensor corruption dramatically degrades performance
   - A simple preparation pipeline (calibration + detrending + filtering) recovers most performance
   - Test with corruption BEFORE buying hardware!

2. **Multi-Rate Sensor Fusion**
   - Different sensors need different staleness policies
   - IMU (fast, noisy) → `'zero'` policy
   - GPS (slow, accurate) → `'hold'` policy
   - Camera (intermittent) → `'drop'` policy
   - Proper fusion can combine complementary sensors effectively

3. **Stress Testing**
   - Multiple corruptions compound (worse than sum of parts)
   - Order matters in preparation pipeline:
     1. Handle dropouts first (staleness policies)
     2. Remove outliers/spikes (median filter, outlier removal)
     3. Calibrate (remove bias)
     4. Detrend (remove drift)
     5. Filter (low-pass, smoothing)
     6. Compensate delay (if known)

### The PyKal Workflow for Hardware Deployment

```
1. Design control system (DynamicalSystem composition)
   ↓
2. Simulate with CLEAN data (verify algorithm works)
   ↓
3. Simulate with CORRUPTED data (THIS NOTEBOOK)
   ↓  → Add realistic sensor corruption
   ↓  → Build preparation pipeline
   ↓  → Verify robustness
   ↓
4. Deploy to hardware (ROSNode)
   ↓  → Same corruption patterns appear
   ↓  → Same preparation pipeline works
   ↓  → System is robust!
```

### Key Takeaways

- **Don't trust simulation** unless it includes realistic corruption
- **Characterize your sensors** (noise, bias, drift, dropout rate)
- **Build a preparation pipeline** BEFORE deployment
- **Test staleness policies** for multi-rate systems
- **Stress test** with multiple corruptions at once
- **Use the same Q matrix** for both `corrupt.with_gaussian_noise()` and your Kalman filter

### Next Steps

1. Characterize YOUR sensors (collect real data, measure noise/bias/dropout rates)
2. Add those corruption parameters to YOUR simulations
3. Build YOUR preparation pipeline
4. Deploy to hardware with confidence

**Hardware will still surprise you. But now you're prepared.**

In [ ]:
print("\n" + "="*60)
print("SIMULATION COMPLETE")
print("="*60)
print("\nYou are now equipped to:")
print("  ✓ Simulate realistic sensor corruption")
print("  ✓ Build preparation pipelines")
print("  ✓ Handle multi-rate sensor fusion")
print("  ✓ Stress test before hardware deployment")
print("\nGo forth and build robots that work in the real world.")
print("="*60)
---

## Navigation

**[Previous: The Curse of Hardware](curse_of_hardware.ipynb)** | **Next: [ROS Deployment →](ros_deployment.ipynb)**

**← [Back to What is PyKal?](../what_is_pykal/index.rst)**